# Large Language Models

## **Session 4-4:** Create Slider-Crank model using an online API
    
Alternatively, instead of using your local resources, various companies provide __APIs__ for their proprietary models. Note that the owner of the key pays on a token basis. The prices shown here were checked on 6.7.2025 and may change. 


| Model | Provider | Input  | Output  |  
| -------- | ------- | :--------: | :--------: |
|     |  | per 1M tokens | per 1M tokens | 
| GPT-4.1  |  OpenAI    | \$ 2.00 | \$ 8.00 |  
| GPT-4.1 mini |  OpenAI    | \$ 0.40 | \$ 1.60 | 
|  o1   | OpenAI | \$15.00 | \$60.00 |
| Gemini Pro 2.5    | Google | \$ 1.25 - 2.50 | \$ 10.00 - 15.00 |
|  Gemini Flash 2.5   | Google | \$ 0.30 | \$ 2.50 |
| Claude Opus 4 | Anthropic | \$ 15.00  | \$ 75.00 |
| Claude Sonnet 4 | Anthropic | \$ 3.00 | \$ 15.00 |


* Gemini Pro: price increases for inputs > __200k tokens__
* OpenAI offers a (more expensive) __real-time__ API and (cheaper) __batched API__ (may take up to 24h, running asynchronously)
* Depending on the provided requests per minute (RPM) and tokens per minute (TPM) might be limited according to the type of contract/subscription.  
* While there are different providers, the following example is using the OpenAI interface 
* To run the code, an api key is needed; this can be done either via writing it into a file
* __NEVER__ upload api key to __public repository__! $\Rightarrow$ Anybody with the key can use the API at your cost.
* [keys](https://platform.openai.com/api-keys), [usage](https://platform.openai.com/usage) and the model's requests can be monitored online. 
* The code is part of the [paper](https://doi.org/10.21203/rs.3.rs-6566994/v1) _Creation, Evaluation and Self-Validation of Simulation Models with Large Language Models_, see also our [github](https://github.com/jgerstmayr/AI-engineering-lab) and the file [processRessultsWithOpenAI_GPT4o](https://github.com/jgerstmayr/AI-engineering-lab/blob/main/processResultsWithOpenAI_GPT4o.py).

<!-- 
Send out file with API keys for workshop.
Talk Möltner: Thursday 11:50 
-->

## Note: 
* To run the following script, an API key is required!
* You do not need to create one yourself
* One key will be sent out shortly before the workshop and deactivated afterwards. 

In [1]:
import os
import time
from LLMHelperFunctions import CheckOutputLLM # helper function
import openai
from openai import OpenAI
from dotenv import load_dotenv

flag_env = load_dotenv("openai_api_key.env")
if flag_env: 
    print('key loaded successfully')
else: 
    print('key could not be loaded - check if it is available!')

key loaded successfully


Initialize the API using the (private) key. The key is a combination of alphanumeric characters similar to an ssh-key.   
It could also be hardcoded as a string in the interface below. The key is created [online](https://platform.openai.com/api-keys).  

In [2]:
# initialize the client using the API key from the environment variable
client = OpenAI(
        api_key= os.environ.get("OPENAI_API_KEY", None),
    )


### Availible models
A list of availibe models is also availible [online](https://platform.openai.com/docs/models/).  
Some models (e.g. 4o) have different input/output modalities. 
The date (e.g. ) refers to the date of the _snapshot_. If the behaviour/performance of a specific model should be consistant the snapshot can be specified. 



In [3]:
modelList = client.models.list().to_dict()
print('model list: ')
for i, data in enumerate(modelList['data']): 
    name_i = data['id']
    print(i, ': ', data['id'])

model list: 
0 :  gpt-4-0613
1 :  gpt-4
2 :  gpt-3.5-turbo
3 :  o4-mini-deep-research-2025-06-26
4 :  codex-mini-latest
5 :  gpt-4o-realtime-preview-2025-06-03
6 :  gpt-4o-audio-preview-2025-06-03
7 :  o4-mini-deep-research
8 :  davinci-002
9 :  babbage-002
10 :  gpt-3.5-turbo-instruct
11 :  gpt-3.5-turbo-instruct-0914
12 :  dall-e-3
13 :  dall-e-2
14 :  gpt-4-1106-preview
15 :  gpt-3.5-turbo-1106
16 :  tts-1-hd
17 :  tts-1-1106
18 :  tts-1-hd-1106
19 :  text-embedding-3-small
20 :  text-embedding-3-large
21 :  gpt-4-0125-preview
22 :  gpt-4-turbo-preview
23 :  gpt-3.5-turbo-0125
24 :  gpt-4-turbo
25 :  gpt-4-turbo-2024-04-09
26 :  gpt-4o
27 :  gpt-4o-2024-05-13
28 :  gpt-4o-mini-2024-07-18
29 :  gpt-4o-mini
30 :  gpt-4o-2024-08-06
31 :  chatgpt-4o-latest
32 :  o1-preview-2024-09-12
33 :  o1-preview
34 :  o1-mini-2024-09-12
35 :  o1-mini
36 :  gpt-4o-realtime-preview-2024-10-01
37 :  gpt-4o-audio-preview-2024-10-01
38 :  gpt-4o-audio-preview
39 :  gpt-4o-realtime-preview
40 :  omni-mod

In [4]:
dt_total, nTokensTotal = 0, 0
with open("ContextGeneral.txt", "r") as file:
    # Read all lines into a list
    lines = file.readlines()
with open("promptSliderCrank.txt", "r") as file:
    myPrompt = file.read()
    
myContext  = "".join(lines[31:])
print(myContext)
print(myPrompt)

#In the following, there are examples to create multibody systems in Exudyn and also important notes about the interface.
#NOTE: mbs.Create...(...) calls several functions in the background to create nodes, objects, markers and loads in Exudyn.
#most quantities such as initial or reference positions and velocities are giving as 3D lists [x,y,z] for positions, velocities, ....
#rotations are usually given as rotation matrix (3x3 numpy array); 
#RotationVector2RotationMatrix([rotX, rotY, rotZ]) computes a rotation around the global x,y,z rotation axis
#for working with rigid bodies, note that there is always a local coordinate system in the body, 
#which can be used to define the location of position and orientation of joints, see the examples.
    
#%%++++++++++++++++++++++++++++++++++++++++++++++++++++
#create rigid bodies and mass points with distance constraint and joints
import exudyn as exu
from exudyn.utilities import * #includes itemInterface and rigidBodyUtilities
import exudyn.

### Create Output
Start inference with one of the shown models.  
Here, the context can be given either as a system prompt or as part of the content.  

In [5]:
t1 = time.time()
response = client.chat.completions.create(
          # model='gpt-4o-2024-08-06',
          # model='o3-mini',
          model = 'gpt-4.1-mini-2025-04-14',
          messages=[
            {"role": "system", "content": myContext},
            {"role": "user", "content":  myPrompt}
          ], 
          store=False, 
        )  
dt = time.time() - t1
responseDict = response.to_dict(mode='json') # choose formatting

### Inspect the Response
The response contains, in addition to the output itself, also a lot of other information: which model was used, type of input, tokens created, etc.  
The output (string) is part of the choices. The tokens/second depend on the model and might fluctuate with the "mini" and "nano" models being faster than the larger ones. 


In [6]:
nTokens =  responseDict['usage']['prompt_tokens']
print('created {} tokens in {} seconds | {} tokens/second\n'.format(nTokens, round(dt, 1), round(nTokens/dt, 2)))

for key, value in dict(response).items(): 
        print(key, ': ', value, '\n')
    

created 2055 tokens in 24.4 seconds | 84.32 tokens/second

id :  chatcmpl-BrREo3FWCSSY4dIL23VeA3VixLUPP 

choices :  [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Based on your request and the previous information, I provide Python code to create a 3-link system of rigid bodies in Exudyn. The parameters are: each rigid body mass = 10 kg, length = 2 m along X, width and height = 0.1 m, gravity acts in -Y direction, first link attached to ground via revolute joint at left end, and a prismatic joint attached to the last link.\n\nHere's the full code with explanations and usage of the provided functions:\n\n```python\nimport exudyn as exu\nfrom exudyn.utilities import *  # for inertia and rotation functions\nimport exudyn.graphics as graphics\nimport numpy as np\nfrom math import pi\n\nSC = exu.SystemContainer()\nmbs = SC.AddSystem()\n\n# Ground plane visualization (optional)\ngGround0 = graphics.CheckerBoard(point=[3,0,-2], normal=[0,0,1], si

Multiple ```choices``` could be part of the response, but in our case it should only be a single one.  

In [7]:
for i in range(len(response.choices)): 
    res = response.choices[i]
    print(res.message.content)
    strOutput = res.message.content

Based on your request and the previous information, I provide Python code to create a 3-link system of rigid bodies in Exudyn. The parameters are: each rigid body mass = 10 kg, length = 2 m along X, width and height = 0.1 m, gravity acts in -Y direction, first link attached to ground via revolute joint at left end, and a prismatic joint attached to the last link.

Here's the full code with explanations and usage of the provided functions:

```python
import exudyn as exu
from exudyn.utilities import *  # for inertia and rotation functions
import exudyn.graphics as graphics
import numpy as np
from math import pi

SC = exu.SystemContainer()
mbs = SC.AddSystem()

# Ground plane visualization (optional)
gGround0 = graphics.CheckerBoard(point=[3,0,-2], normal=[0,0,1], size=10)
oGround = mbs.CreateGround(graphicsDataList=[gGround0])

# Rigid body parameters
mass = 10         # kg
length = 2        # m, along X-axis
width = 0.1       # m (Z direction)
height = 0.1      # m (Y direction)

# Cal

### Extract Code
The model (usually) not only outputs the code itself, but also some explanation before and/or afterwards.  
The code itself should be put between the ```python``` tag by the model as shown below, although not all models might follow this convention. 


In [8]:
flagExtractedCode = False
try: 
    code = strOutput.split("```python")[1].split("```")[0]
    flagExtractedCode = True 
except: 
    print('Model did not use ```python to mark code. Trying "python3". ')

if not(flagExtractedCode): 
    try: 
        code = strOutput.split("python3")[1].split("--------------------------")[0]
        flagExtractedCode = True
    except: 
        print('Model did not use python3 mark either.')

CheckOutputLLM(code)

if flagExtractedCode: 
    exec(code)
else: 
    print('code could not be extracted from model output')

columns imported = [21, 21, 21, 0, 0, 23, 0]
total columns to be imported = 86 , array size of file = 87


Note: A common error of the LLM in this task is to _hallucinate_ color options which are not available, for example _skyblue_. 
Optional task: change the prompt to create a more _meaningful_ mechanism.  